In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
import importlib
import argparse
import time

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.autograd as autograd
from torch.autograd import Variable
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision import transforms
import torchvision.utils as vutils
from torchnet.meter import AverageValueMeter

In [20]:
parser = {
    'dataset': 'cifar10',
    'dataroot': './data',
    'workers': 2,
    'batch_size': 4,
    'image_size': 28,
    'image_channels': 3,
    'nz': 100, #dim of latent variables
    'ngf': 64, #number of G features
    'ndf': 64, #number of D features
    'epochs': 25, 
    'lr': 1e-3,
    'beta1': 0.5, #for adam opt
    'netG': '',
    'netD': '',
    'outf': './output',
#    'ngpu': 0,
    'manualSeed': 7,
    'no_cuda': True,
}
args = argparse.Namespace(**parser)

# Load data

In [17]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
train_set = dset.MNIST(root=args.dataroot, train=True, transform=trans, download=True)
test_set = dset.MNIST(root=args.dataroot, train=False, transform=trans)

In [18]:
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=args.batch_size, shuffle=True, num_workers=args.workers)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=args.batch_size, shuffle=False, num_workers=args.workers)

In [19]:
for i, j in train_loader:
    print (i)
    print (j)
    break


(0 ,0 ,.,.) = 
 -0.5000 -0.5000 -0.5000  ...  -0.5000 -0.5000 -0.5000
 -0.5000 -0.5000 -0.5000  ...  -0.5000 -0.5000 -0.5000
 -0.5000 -0.5000 -0.5000  ...  -0.5000 -0.5000 -0.5000
           ...             ⋱             ...          
 -0.5000 -0.5000 -0.5000  ...  -0.5000 -0.5000 -0.5000
 -0.5000 -0.5000 -0.5000  ...  -0.5000 -0.5000 -0.5000
 -0.5000 -0.5000 -0.5000  ...  -0.5000 -0.5000 -0.5000
     ⋮ 

(1 ,0 ,.,.) = 
 -0.5000 -0.5000 -0.5000  ...  -0.5000 -0.5000 -0.5000
 -0.5000 -0.5000 -0.5000  ...  -0.5000 -0.5000 -0.5000
 -0.5000 -0.5000 -0.5000  ...  -0.5000 -0.5000 -0.5000
           ...             ⋱             ...          
 -0.5000 -0.5000 -0.5000  ...  -0.5000 -0.5000 -0.5000
 -0.5000 -0.5000 -0.5000  ...  -0.5000 -0.5000 -0.5000
 -0.5000 -0.5000 -0.5000  ...  -0.5000 -0.5000 -0.5000
     ⋮ 

(2 ,0 ,.,.) = 
 -0.5000 -0.5000 -0.5000  ...  -0.5000 -0.5000 -0.5000
 -0.5000 -0.5000 -0.5000  ...  -0.5000 -0.5000 -0.5000
 -0.5000 -0.5000 -0.5000  ...  -0.5000 -0.5000 -0.5000
 

In [ ]:
mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)
mb_size = 64
Z_dim = 100
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
h_dim = 128
cnt = 0
lr = 1e-3

To do:
- Get one hot encode label y

# Define class

## Generator

## Discriminator

In [ ]:
#Define class Generator:
class _netG(nn.Module):
    #def __init__(self, ngpu):
    def __init__(self):
        super(_netG, self).__init__()  #no need to list __init__ of nn.Module
        #self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(args.nz, args.ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(args.ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(args.ngf * 8, args.ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(args.ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(args.ngf * 4, args.ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(args.ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(args.ngf * 2, args.ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(args.ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(args.ngf, args.image_channels, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64 --> image size = 64
        )

    def forward(self, input):
        #gpu_ids = None
        #if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            #gpu_ids = range(self.ngpu)
        #return nn.parallel.data_parallel(self.main, input, gpu_ids)
        return self.main(input)